In [32]:
from sly import Lexer, Parser
import sys

In [33]:
#Tablas donde ira todo:
tabla = {}
tablaEnsam = {}
#Bandera identificacion:
banderaDIGI = False
banderaID = False
#Bandera para inicializacion:
numVariables = 0
#Etiqueta final:
final = 0
false = 0
#Operaciones:
numOperadores = 0
#Ver si x operador es ID o NUMERO:
Numa = False #Si True, emptya es un id y no un numero.
Numb = False #Si True, emptyb es un id y no un numero.

In [34]:
class NodoPrint():
    def P(a):
        if a in tabla:
            pass
        else:
            print("Syntax error: ID '%c' isnt in table." %a)

class NodoDeclaracion():
    def P(a):
        global numVariables
        numVariables = numVariables+1
        tablaEnsam[a] = -numVariables*4
        tabla[a] = None

class NodoID():
    def P(a,b):
        global banderaIF, banderaExpr, Numa, numOperadores
        if Numa == True:
            if numOperadores > 1:
                print('movl %eax, {}(%ebp)'.format(tablaEnsam[a]))
            else:
                print('movl {}(%ebp), {}(%ebp)'.format(tablaEnsam[b],tablaEnsam[a]))
        else:
            if numOperadores > 1:
                print('movl %eax, {}(%ebp)'.format(tablaEnsam[a]))
            else:
                print('movl ${}, {}(%ebp)'.format(b,tablaEnsam[a]))
        print('')

class NodoGen():
    def P(a):
        return a  

class NodoCmpl():
    def P(a,b):
        global Numa, Numb
        if Numa == True and Numb == True:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
            print('cmpl {}(%ebp), %eax'.format(tablaEnsam[b]))
        if Numa == False and Numb == True:
            print('movl ${}, %eax'.format(a))
            print('cmpl {}(%ebp), %eax'.format(tablaEnsam[b]))
        if Numa == True and Numb == False:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
            print('cmpl ${}, %eax'.format(b))          
        if Numa == False and Numb == False:
            print('movl ${}, %eax'.format(a))
            print('cmpl ${}, %eax'.format(b))           
        
class NodoMenor():
    def P(a,b):
        global final
        print('jl final{}'.format(final))
        print('')

class NodoMayor():
    def P(a,b):
        global final
        print('jg final{}'.format(final))
        print('')
    
class NodoMenorIg():
    def P(a,b):
        global final
        print('jle final{}'.format(final))
        print('')
        
class NodoMayorIg():
    def P(a,b):
        global final
        print('jge final{}'.format(final))
        print('')
        
class NodoIg():
    def P(a,b):
        global final
        print('je final{}'.format(final))
        print('')
    
class NodoAnd():
    def P(a,b):
        global final
        print('compl $0, %eax')
        print('je final{}'.format(final))
        print('')
    
class NodoOr():
    def P(a,b):
        global final
        print('compl $0, %eax')
        print('jne final{}'.format(final))        
        print('')
        
class NodoNot():
    def P(a):
        global final
        print('compl $0, %eax')
        print('je final{}'.format(final))
        print('movl $1, %eax')
        print('')
    
class NodoSum():
    def P(a,b):
        global Numa, Numb
        if Numa == True and Numb == True:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
            print('addl {}(%ebp), %eax'.format(tablaEnsam[b]))
            #El resultado se guarda en %eax.
        if Numa == True and Numb == False:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
            print('addl ${}, %eax'.format(b))
        if Numa == False and Numb == True:
            print('movl ${}, %eax'.format(a))
            print('addl {}(%ebp), %eax'.format(tablaEnsam[b]))                  
        if Numa == False and Numb == False:
            print('movl ${}, %eax'.format(a))
            print('addl ${}, %eax'.format(b))             
        print('')
        return a
        
class NodoResta():
    def P(a,b):
        print('movl {}(%ebp), %eax'.format(a))
        print('subl {}(%ebp), %eax'.format(b))
        #El resultado se guarda en %eax.
        print('')
        return a
    
class NodoMul():
    def P(a,b):
        print('movl {}(%ebp), %eax'.format(a))
        print('imull {}(%ebp), %eax'.format(b))
        #El resultado se guarda en %eax.
        print('')
        return a
    
class NodoDiv():
    def P(a,b):
        global Numa, Numb
        print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
        print('cdq')
        print('movl {}(%ebp), %ebx'.format(tablaEnsam[b]))
        print('divl %ebx')
        #Lo mismo, el propio resultado se guarda en %eax.
        print('')
        return a
    
class NodoRU():
    def P(a):
        global Numa
        print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
        print('imull -1, %eax')
        #No mover el resultado intermedio a %eax, solo moverlo al final, en la asignacion.
        return a
    
class NodoTabla():
    def P(a):
        return a

In [35]:
class CalcLexer(Lexer):
    tokens = {DIGIT, ZERO, ID, LESEQ, BIGEQ, EQ, PRINTF, PGENERIC, IF , ELSE, WHILE, MAIN, RETURN, INT}
    ignore = ' \t\n'
    ZERO = r'0'
    DIGIT = r'[0-9]+'
    ELSE = r'else'
    PRINTF = r'printf'
    PGENERIC = r'%d'
    INT = r'int'
    MAIN = r'main'
    IF = r'if'
    WHILE = r'while'
    RETURN = r'return'
    ID = r'[a-zA-Z][a-zA-Z0-9]*'
    
    LESEQ = r'<='
    BIGEQ = r'>='
    EQ = r'=='
    
    literals = {'+', '-', '*', '/', '(', ')', '<', '>', '=', ';', '|', '&', '~', ',', '"', '{', '}'}
    
    @_(r'0')
    def ZERO(self, f):
        f.value=int(f.value)
        return f
    
    @_(r'[0-9]+')
    def DIGIT(self, f):
        f.value=int(f.value)
        return f
    
    def ID(self, f):
        f.value=str(f.value)
        return f
    
    def error(self, t):
        print('Bad character %r' % t.value[0])
        sys.exit(0)    

In [36]:
class CalcParser(Parser):
    tokens=CalcLexer.tokens
    def __init__(self):
        self.names={}
    
    @_('INT MAIN "(" ")" "{" entradaDeclaraciones returnS  "}"')
    def entradaPrincipal(self, p):
        pass
    
    @_('INT ID emptyID masDeclaraciones ";" entradaDeclaraciones')
    def entradaDeclaraciones(self, p):
        pass
    
    @_(' "," ID emptyID masDeclaraciones')
    def masDeclaraciones(self, p):
        pass
    
    @_('')
    def intro(self, p):
        print('.text')
        print('.globl main')
        print('.type main, @function')
        print('main:')
        print('')
        print('push %ebp')
        print('movl %esp, %ebp')
        print('subl ${}, %esp'.format(len(tablaEnsam)*4))
        print('')
        pass
    
    @_('')
    def emptyID(self, p):
        NodoDeclaracion.P(p[-1])
        pass
    
    @_('')
    def masDeclaraciones(self, p):
        pass
    
    @_('intro entrada')
    def entradaDeclaraciones(self, p):
        pass
    
    @_('entrada datos')
    def entrada(self, p):
        pass
    
    @_('entrada sentenciaIF')
    def entrada(self, p):
        pass
    
    @_('entrada sentenciaWHILE')
    def entrada(self, p):
        pass
    
    @_('')
    def entrada(self, p):
        pass
    
    @_('WHILE "(" expr emptyWHILE ")" "{" entrada "}"')
    def sentenciaWHILE(self, p):
        global banderWHILE
        bandereaWHILE = False
        pass
    
    @_('')
    def emptyWHILE(self, p):
        global banderaWHILE, banderaExpr
        banderaWHILE = True
        banderaExpr = p[-1]
        pass
    
    @_('IF "(" expr ")" "{" entrada "}"')
    def sentenciaIF(self, p):
        global final
        print('final{}:'.format(final))
        final=final+1
        print('')
        pass
        
    @_('IF "(" expr ")" "{" entrada "}" emptyIfElse elseS')
    def sentenciaIF(self, p):
        global final
        print('final{}:'.format(final)) #final1:
        final=final+1
        pass
    
    @_('ELSE  "{" entrada "}"')
    def elseS(self, p):
        pass
    
    @_('')
    def emptyIfElse(self, p):
        global final
        final=final+1
        print('jmp final{}'.format(final)) #jmp final1
        final=final-1
        print('final{}:'.format(final)) #final0:
        final=final+1
        pass
    
    @_('PRINTF "(" """ PGENERIC mas ID ")" ";" ')
    def datos(self, p):
            NodoPrint.P(p.ID)
            
    @_('""" ","')
    def mas(self,p):
        pass
    
    @_('PGENERIC mas ID ","')
    def mas(self ,p):
        NodoPrint.P(p.ID)        
        
    @_('ID "=" expr emptya ";"')
    def datos(self, p):
        global numOperadores
        NodoID.P(p.ID, p.expr)
        numOperadores = 0
        pass
        
    @_('expr emptya "<" comp emptyb')
    def expr(self, p):
        NodoCmpl.P(p.expr, p.comp)
        NodoMenor.P(p.expr, p.comp)
        pass
    
    @_('expr emptya ">" comp emptyb')
    def expr(self, p):
        NodoCmpl.P(p.expr, p.comp)
        NodeoMayor.P(p.expr, p.comp)
        pass
    
    @_('expr emptya LESEQ comp emptyb')
    def expr(self, p):
        NodoCmpl.P(p.expr, p.comp)
        NodoMenorIg.P(p.expr, p.comp)
        pass
    
    @_('expr emptya BIGEQ comp emptyb')
    def expr(self, p):
        NodoCmpl.P(p.expr, p.comp)
        NodoMayorIg.P(p.expr, p.comp)
        pass
    
    @_('expr emptya EQ comp emptyb')
    def expr(self, p):
        NodoCmpl.P(p.expr, p.comp)
        NodoIg.P(p.expr, p.comp)
        pass
    
    @_('expr "&" comp')
    def expr(self, p):
        NodoAnd.P(p.expr, p.comp)
        pass
    
    @_('expr "|" comp')
    def expr(self, p):
        NodoOr.P(p.expr, p.comp)
        pass      
    
    @_('"~" expr')
    def expr(self, p):
        NodoNot.P(p.expr)
        pass
      
    @_('comp')
    def expr(self, p):
        return NodoGen.P(p.comp)
    
    @_('comp emptya "+" sum emptyb')
    def comp(self, p):
        return NodoSum.P(p.comp, p.sum)
        
    
    @_('comp emptya "-" sum emptyb')
    def comp(self, p):
        return NodoResta.P(p.comp, p.sum)
    
    @_('sum')
    def comp(self, p):
        return NodoGen.P(p.sum)
    
    @_('sum emptya "*" fact emptyb')
    def sum(self, p):
        return NodoMul.P(p.sum, p.fact)
    
    @_('sum emptya "/" fact emptyb')
    def sum(self, p):
        return NodoDiv.P(p.sum, p.fact)
        
    @_('fact')
    def sum(self, p):
        return NodoGen.P(p.fact)
    
    @_('"-" fact emptya')
    def fact(self, p):
        return NodoRU.P(p.fact)
    
    @_('')
    def emptya(self, p):
        global Numa
        if p[-1] in tablaEnsam:
            Numa = True
        else:
            Numa = False
            
    @_('')
    def emptyb(self, p):
        global Numb
        if p[-1] in tablaEnsam:
            Numb = True
        else:
            Numb = False
    
    @_('ID emptyc')
    def fact(self, p):
        return NodoTabla.P(p.ID)

    @_('DIGIT emptyc')
    def fact(self, p):
        return NodoGen.P(p.DIGIT)
    
    @_('')
    def emptyc(self, p):
        global numOperadores
        numOperadores=numOperadores+1
    
    @_('"(" expr ")"')
    def fact(self, p):
        return NodoGen.P(p.expr)
    
    @_('RETURN ZERO ";"')
    def returnS(self, p):
        print('movl $0, %eax')
        print('')
        print('movl %ebp, %esp')
        print('popl %ebp')
        print('ret')
        print('')
        pass

In [37]:
if __name__=='__main__':
    lexer=CalcLexer() 
    parser=CalcParser()
    data='int main(){int a,b,c; a=1; b=1; c=a; if(a==1){b=3;} if(a<2){b=4;}else{a=3;} return 0;}'
    parser.parse(lexer.tokenize(data))

.text
.globl main
.type main, @function
main:

push %ebp
movl %esp, %ebp
subl $12, %esp

movl $1, -4(%ebp)

movl $1, -8(%ebp)

movl -4(%ebp), -12(%ebp)

movl -4(%ebp), %eax
cmpl $1, %eax
je final0

movl %eax, -8(%ebp)

final0:

movl -4(%ebp), %eax
cmpl $2, %eax
jl final1

movl %eax, -8(%ebp)

jmp final2
final1:
movl $3, -4(%ebp)

final2:
movl $0, %eax

movl %ebp, %esp
popl %ebp
ret



In [9]:
data=open("Prueba.c", "r")
for x in data:
    print(x)

int main(){

	int a,b,c;

	a=1;

	b=2;

	c=3;

	if(a==1){

		b=3;

	}

	if(a==2){

		b=4;

	}else{

		b=5;

	}

	return 0;

}
